# 6.1 训练工具介绍

In [1]:
# HuggingFace提供了巨大的模型库，但这些模型往往是在广义的数据集上训练的，缺乏针对特定
# 数据集的优化，所以在获得一个合适的模型之后，往往还要针对具体
# 任务的特定数据集进行二次训练，这就是所谓的迁移学习。

# HuggingFace提供了训练工具，统一了模型的再训练过程，使调
# 用者无须了解具体模型的计算过程，只需针对具体的任务准备好数据
# 集，便可以再训练模型。

# 在本章中将使用一个情感分类任务的例子来再训练一个模型，以
# 此来讲解HuggingFace训练工具的使用方法。

# 6.2 使用训练工具

### 6.2.1 准备数据集

###### 1.加载编码工具

In [2]:
# 首先加载一个编码工具，由于编码工具和模型往往是成对使用的，
# 所以此处使用hfl/rbt3编码工具，因为要再训练的模型是hfl/rbt3
# 模型，代码如下：
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')
from transformers import Trainer
from transformers.data.data_collator import DataCollatorWithPadding
from datasets import load_from_disk
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification
import torch
from datasets import load_metric
# import evaluate
import numpy as np
from transformers.trainer_utils import EvalPrediction
from transformers import TrainingArguments

In [3]:
# 加载了编码工具之后不妨试算一下，观察一下输出，代码如下：
#第6章/试编码句子
tokenizer.batch_encode_plus(
['明月装饰了你的窗子', '你装饰了别人的梦'],
truncation=True,
)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 2094, 102], [101, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

###### 2.准备数据集

In [4]:
# 加载数据集，使用该数据集来再训练模型，代码如下：
#第6章/从磁盘加载数据集
# 在这段代码中，对数据集进行了采样，目的有以下两方面：一是
# 便于测试；二是模拟再训练集的体量较小的情况，以验证即使是小的
# 数据集，也能通过迁移学习得到一个较好的训练结果。

dataset = load_from_disk('./data/ChnSentiCorp')
#缩小数据规模，便于测试
dataset['train'] =dataset['train'].shuffle().select(range(2000))
dataset['test'] = dataset['test'].shuffle().select(range(100))
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [5]:
# 现在的数据集还是文本数据，使用编码工具把这些抽象的文字编
# 码成计算机善于处理的数字
#第6章/编码

def f(data):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')
    return tokenizer.batch_encode_plus(data['text'],truncation=True)
   

dataset=dataset.map(function=f,
batched=True,
batch_size=1000,
num_proc=4,
remove_columns=['text'])

dataset


Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [6]:
# 可以看到，原本数据集中的text字段已经被移除，但多了
# input_ids、token_type_ids、attention_mask字段，这些字段是编
# 码工具编码的结果，这和前面观察到的编码器试算的结果一致。

In [7]:
# 由于模型对句子的长度有限制，不能处理长度超过512个词的句
# 子，所以需要把数据集中长度超过512个词的句子过滤掉，代码如
# 下：

In [8]:
#第6章/移除太长的句子
def f(data):
    return [len(i)<=512 for i in data['input_ids']]
dataset=dataset.filter(f, batched=True, batch_size=1000,
num_proc=4)
dataset


Filter (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1981
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1190
    })
})

In [9]:
# 注意：对于数据长度超过模型限制有很多处理方法，此处只演示
# 了最简单的丢弃法。也可以把超出长度的部分截断，留下符合模型长
# 度要求的数据，截断数据时可以截断数据的尾部，也可以截断数据的
# 头部，当截断数据时，编码结果中的input_ids、token_type_ids、
# attention_mask要一起截断，因为它们是一一对应的关系。


###### 6.2.2 定义模型和训练工具

### 1.加载预训练模型

In [10]:
#数据集准备好了，现在就可以加载要再训练的模型了，代码如下：
#第6章/加载模型

model=AutoModelForSequenceClassification.from_pretrained('hfl/rbt3',num_labels=2)
#统计模型参数量
sum([i.nelement() for i in model.parameters()]) / 10000

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3847.8338

如前所述，此处加载的模型应该和编码工具配对使用，所以此处
加载的模型为hfl/rbt3模型，该模型由哈尔滨工业大学讯飞联合实验
室(HFL)分享到HuggingFace模型库，这是一个基于中文文本数据训
练的BERT模型。后续将使用准备好的数据集对该模型进行再训练，在
代码的最后一行统计了该模型的参数量，以大致衡量一个模型的体量
大小。该模型的参数量约为3800万个，这是一个较小的模型。

加载了模型之后，不妨对模型进行一次试算，以观察模型的输
出，代码如下：

In [13]:
#第6章/模型试算
#模拟一批数据
data = {
'input_ids': torch.ones(4, 10, dtype=torch.long),
'token_type_ids': torch.ones(4, 10, dtype=torch.long),
'attention_mask': torch.ones(4, 10, dtype=torch.long),
'labels': torch.ones(4, dtype=torch.long)
}
#模型试算
out = model(**data)
out['loss'], out['logits'].shape

(tensor(0.4981, grad_fn=<NllLossBackward0>), torch.Size([4, 2]))

### 2.定义评价函数

为了便于在训练过程中观察模型的性能变化，需要定义一个评价
指标函数。对于情感分类任务往往关注正确率指标，所以此处加载正
确率评价函数，代码如下：

In [14]:
#第6章/加载评价指标


metric = load_metric('accuracy')
# metric2 = evaluate.load('accuracy')

C:\Users\Miao Jiangbei\AppData\Local\Temp\ipykernel_28560\1499779853.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')
C:\Users\Miao Jiangbei\AppData\Roaming\Python\Python39\site-packages\datasets\load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


由于模型计算的输出和评价指标要求的输入还有差别，所以需要
定义一个转换函数，把模型计算的输出转换成评价指标可以计算的数
据类型，这个函数就是在训练过程中真正要用到的评价函数，代码如
下

In [15]:
#第6章/定义评价函数

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.argmax(axis=1)
    return metric.compute(predictions=logits, references=labels)

#模拟输出
eval_pred = EvalPrediction(
predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
label_ids=np.array([1, 1, 0, 1]),
)
compute_metrics(eval_pred)

{'accuracy': 0.75}

In [16]:
# def compute_metrics2(eval_pred):
#     logits, labels = eval_pred
#     logits = logits.argmax(axis=1)
#     return metric2.compute(predictions=logits, references=labels)
# eval_pred = EvalPrediction(
# predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
# label_ids=np.array([1, 1, 0, 1]),
# )
# compute_metrics(eval_pred)

### 3.定义训练超参数

##### 在开始训练之前，需要定义好超参数，HuggingFace 使用 `TrainingArguments`对象来封装超参数，代码如下：


In [17]:
#第6章/定义训练参数

#定义训练参数
args = TrainingArguments(
#定义临时数据保存路径
    output_dir='./output_dir',
#定义测试执行的策略，可取值为no、epoch、steps
    evaluation_strategy='steps',
#定义每隔多少个step执行一次测试
    eval_steps=30,
#定义模型保存策略，可取值为no、epoch、steps
    save_strategy='steps',
#定义每隔多少个step保存一次
    save_steps=30,
#定义共训练几个轮次
    num_train_epochs=1,
#定义学习率
    learning_rate=1e-4,
#加入参数权重衰减，防止过拟合
    weight_decay=1e-2,
#定义测试和训练时的批次大小
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
#定义是否要使用GPU训练
    use_cpu=True
)


In [18]:
# 对于初学者建议从这些简单的参数开始调试，完整的参数列表可参照HuggingFace官方文档。


### 4.定义训练器


In [19]:
#第6章/定义训练器

#定义训练器
# 定义训练器时需要传递要训练的模型、超参数对象、训练和验证
# 数据集、评价函数，以及数据整理函数。
trainer = Trainer(
model=model,
args=args,
train_dataset=dataset['train'],
eval_dataset=dataset['test'],
compute_metrics=compute_metrics,
data_collator=DataCollatorWithPadding(tokenizer),
)


### 5.数据整理函数介绍

###### 数据整理函数使用了由 HuggingFace 提供的 `DataCollatorWithPadding` 对象，
它能把一个批次中长短不一的句子补充成统一的长度，长度取决于这个批次中最长的句子有多长，
所有数据的长度一致后即可转换成矩阵，模型期待的数据类型也是矩阵，
所以经过数据整理函数的处理之后，数据即被整理成模型可以直接计算的矩阵格式。
可以通过下面的例子验证，代码如下：

In [20]:
#第6章/测试数据整理函数
data_collator = DataCollatorWithPadding(tokenizer)
#获取一批数据
data = dataset['train'][:5]
#输出这些句子的长度
for i in data['input_ids']:
    print(len(i))
#调用数据整理函数
data = data_collator(data)
#查看整理后的数据
for k, v in data.items():
    print(k, v.shape)


186
43
166
91
94
input_ids torch.Size([5, 186])
token_type_ids torch.Size([5, 186])
attention_mask torch.Size([5, 186])
labels torch.Size([5])


在这段代码中，首先初始化了一个DataCollatorWithPadding对象作为数据整理函数，
然后从训练集中获取了5条数据作为一批数据，从输出可以看出这些句子有长有短，
之后使用数据整理函数处理这批数据，得到的结果再输出形状，
可以看到这些数据已经被整理成统一的长度，长度取决于这批句子中最长的句子，并且被转换为矩阵形式。

通过如下代码可以查看数据整理函数是如何对句子进行补长的


In [21]:
tokenizer.decode(data['input_ids'][1])


'[CLS] 地 理 位 置 不 错 ， 离 去 机 场 大 大 巴 站 很 近 ， 周 围 吃 饭 很 方 便 ， 有 一 家 齐 齐 火 锅 不 错 ， 也 很 便 宜. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

###### 6.2.3 训练和测试

### 1.训练模型

在开始训练之前，不妨直接对模型进行一次测试，先定下训练前的基准，在训练结束后再对比这里得到的基准，以验证训练的有效性，代码如下：


In [22]:
#评价模型
trainer.evaluate()

{'eval_loss': 0.7311601042747498,
 'eval_accuracy': 0.4444444444444444,
 'eval_runtime': 12.7081,
 'eval_samples_per_second': 7.79,
 'eval_steps_per_second': 0.551}

可见模型在训练之前，有44%的正确率。由于使用的训练集为二
分类数据集，所以44%的正确率近乎于瞎猜。这符合预期，因为模型
还没有训练，接下来对模型进行训练，期待它能超过此处得到的成
绩。

对模型进行训练，代码如下：

In [23]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
30,No log,0.697027,0.696970
60,No log,0.425063,0.838384
90,No log,0.364491,0.858586
120,No log,0.345274,0.868687


TrainOutput(global_step=124, training_loss=0.4012822489584646, metrics={'train_runtime': 699.6392, 'train_samples_per_second': 2.831, 'train_steps_per_second': 0.177, 'total_flos': 71545310026908.0, 'train_loss': 0.4012822489584646, 'epoch': 1.0})

如果在训练的过程中由于各种原因导致训练中断，或者希望从某
个 检 查 点 重 新 训 练 模 型 ， 则 可 以 使 用 训 练 器 的 train() 函 数 的
`resume_from_checkpoint`参数设定检查点，从该检查点重新训练，
代码如下：

In [ ]:
#第6章/从某个存档文件继续训练
trainer.train(resume_from_checkpoint='./output_dir/checkpoint-90')


继续训练和从头训练的输出是一致的，只是继续训练会跳过前90
个steps，所以上面的代码只会训练124−90=34个steps，继续训练
同样会保存检查点，所以上面的代码会覆盖检查点checkpoint-120。


In [24]:
#第6章/评价模型
trainer.evaluate()

{'eval_loss': 0.34446045756340027,
 'eval_accuracy': 0.8686868686868687,
 'eval_runtime': 13.0395,
 'eval_samples_per_second': 7.592,
 'eval_steps_per_second': 0.537,
 'epoch': 1.0}

### 2.模型的保存和加载

训练得到满意的模型之后，可以手动将该模型的参数保存到磁盘
上，以备以后需要时加载，代码如下：

In [27]:
#第6章/手动保存模型参数
trainer.save_model(output_dir='./output_dir/save_model')

加载模型参数的方法如下：


In [30]:
#第6章/手动加载模型参数
import torch
# model.load_state_dict(torch.load('./output_dir/save_model/PyTorch_model.bin'))
model = model.from_pretrained('./output_dir/save_model')

### 3.使用模型预测

In [31]:
#第6章/测试
model.eval()
for i, data in enumerate(trainer.get_eval_dataloader()):
    break
out = model(**data)
out = out['logits'].argmax(dim=1)
for i in range(8):
    print(tokenizer.decode(data['input_ids'][i],skip_special_tokens =True))
    print('label=', data['labels'][i].item())
    print('predict=', out[i].item())

这 次 入 住 和 上 次 相 比, 感 觉 差 很 多 啊... 卫 生 间 马 桶 一 直 漏 水, 早 上 打 扫 房 间 服 务 员 简 直 像 抄 家 一 样, 搞 的 整 个 楼 层 都 鸡 犬 不 宁, 想 多 睡 一 会 儿 都 不 行 ; 晚 上 2 层 的 ktv 音 响 一 直 闹 到 半 夜, 住 在 3 层 的 我 不 但 没 法 睡 觉, 连 听 电 视 的 声 音 都 有 困 难. 下 次 再 去 扬 州, 我 肯 定 要 换 个 酒 店 了.
label= 0
predict= 0
一 进 房 间 就 有 一 股 霉 味 ， 后 来 发 现 马 桶 水 箱 出 不 了 水 ， 两 次 让 服 务 员 来 修 理 ， 还 是 不 太 放 心 ， 而 且 对 服 务 员 的 态 度 也 感 到 不 满 ， 于 是 在 前 台 要 求 更 换 另 一 楼 层 的 房 间 ， 可 是 换 过 了 的 房 间 ， 可 能 很 长 时 间 无 人 住 过 ， 下 水 渠 散 出 很 浓 的 臭 味 ， 真 让 人 不 适 ， 还 好 ， 这 层 的 服 务 员 还 算 很 认 真 地 用 水 冲 洗 了 浴 室 ， 到 了 晚 上 我 们 洗 澡 后 ， 又 发 现 有 水 从 床 前 的 木 地 板 夹 逢 渗 透 出 来 ， 相 对 这 样 收 费 却 又 如 此 的 质 量 ， 真 让 我 无 话 可 说 了 。
label= 0
predict= 0
我 本 身 就 是 一 个 漫 画 迷 ！ 非 常 喜 欢 朱 德 庸 的 作 品 ！ 《 绝 对 小 孩 》 这 本 书 出 版 以 后 立 即 买 回 来 。 里 面 的 内 容 太 搞 笑 了 ！ 女 儿 也 特 别 喜 欢 ， 每 天 回 来 就 赶 紧 写 作 业 ， 然 后 好 看 书 。 哈 哈 ！ 这 是 我 的 小 计 策 ！ 早 点 写 完 作 业 ， 就 可 以 看 这 本 漫 画 书 作 为 奖 励 ！ 《 绝 对 小 孩 2 》 是 作 为 期 末 考 试 95 分 以 上 的 奖 励 买 回 来 的 。 用 这 书 作 为 物 质 刺 激 还 真 是 不 错 呢 ！
label= 1
predict= 1
虽 然 我 的 键 盘 有 些 小 的 问 题 ， 但 这